In [ ]:
import numpy as np
import pandas as pd
# !pip install rasterio
import rasterio as rio
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , classification_report




In [ ]:
from google.colab import drive
import os


data_dir = "/content/drive/MyDrive/Colab Notebooks/EuroSAT"
csv_path = os.path.join(data_dir, "train.csv")
train_csv = "/content/drive/MyDrive/Colab Notebooks/EuroSAT/train.csv"
# test_csv  = "/content/drive/MyDrive/Colab Notebooks/EuroSAT/test.csv"
# val_csv   = "/content/drive/MyDrive/Colab Notebooks/EuroSAT/validate.csv"

df = pd.read_csv(train_csv)
print(df.head())



   Unnamed: 0                                           Filename  Label  \
0       16257                      AnnualCrop/AnnualCrop_142.jpg      0   
1        3297  HerbaceousVegetation/HerbaceousVegetation_2835...      2   
2       17881               PermanentCrop/PermanentCrop_1073.jpg      6   
3        2223                      Industrial/Industrial_453.jpg      4   
4        4887  HerbaceousVegetation/HerbaceousVegetation_1810...      2   

              ClassName  
0            AnnualCrop  
1  HerbaceousVegetation  
2         PermanentCrop  
3            Industrial  
4  HerbaceousVegetation  


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input


data_dir = "/content/drive/MyDrive/Colab Notebooks/EuroSAT"
forest_path = os.path.join(data_dir, "Forest", img_rel)
deforest_path = os.path.join(data_dir, "Deforested", img_rel)

df['BinaryLabel'] = df['ClassName'].apply(
    lambda x: 0 if x == "Forest" else 1
)

# Using CNN preinstaled feature extractoe
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(128,128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x, verbose=0)
    return features.flatten()

X, y = [], []

for idx, row in df.iterrows():
    img_rel = row['Filename']



    if os.path.exists(forest_path):
        img_path = forest_path
    elif os.path.exists(deforest_path):
        img_path = deforest_path
    else:
        print("Missing:", img_rel)
        continue
    feat = extract_features(img_path)
    X.append(feat)
    y.append(row['Label'])

X = np.array(X)
y = np.array(y)

print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)


/tmp/ipython-input-932043826.py:11: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


Feature matrix shape: (18900, 1280)
Labels shape: (18900,)


In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input



data_dir = "/content/drive/MyDrive/Colab Notebooks/EuroSAT"

# Binary labels: 0=Forest, 1=Deforested
df['BinaryLabel'] = df['ClassName'].apply(lambda x: 0 if x == "Forest" else 1)

# Here we are reducing dataset
df = df.sample(2000, random_state=42).reset_index(drop=True)
print("Using subset size:", len(df))


#  Feature extraction going on

base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(128,128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return base_model.predict(x, verbose=0).flatten()


#looping through rows

X, y = [], []

for idx, row in df.iterrows():
    img_rel = row['Filename']

    # Fixing  path logic: don't duplicate "Forest"
    if row['ClassName'] == "Forest":
        img_path = os.path.join(data_dir, img_rel)
    else:
        img_path = os.path.join(data_dir, "Deforested", img_rel)

    if not os.path.exists(img_path):
        print("Missing:", img_path)
        continue

    feat = extract_features(img_path)
    X.append(feat)
    y.append(row['BinaryLabel'])

X = np.array(X)
y = np.array(y)

print(" Features extracted")
print("Feature matrix shape:", X.shape)
print("Labels shape:", y.shape)
print("Unique binary labels:", np.unique(y))




Using subset size: 2000


/tmp/ipython-input-2612510847.py:21: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


✅ Features extracted
Feature matrix shape: (2000, 1280)
Labels shape: (2000,)
Unique binary labels: [0 1]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


clf_stage1 = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf_stage1.fit(X_train, y_train)

y_pred = clf_stage1.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9575

Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.66      0.78        47
           1       0.96      1.00      0.98       353

    accuracy                           0.96       400
   macro avg       0.96      0.83      0.88       400
weighted avg       0.96      0.96      0.95       400


Confusion Matrix:
 [[ 31  16]
 [  1 352]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


#  Subtype classification (only on deforested)


# Filter only deforested samples
mask_deforested = (y == 1)   # binary label = 1 means deforested
X_def = X[mask_deforested]
y_def = df.loc[mask_deforested, "Label"].values

print("Deforested feature matrix shape:", X_def.shape)
print("Subtype labels shape:", y_def.shape)
print("Unique subtypes:", np.unique(y_def))


X_train, X_test, y_train, y_test = train_test_split(
    X_def, y_def, test_size=0.2, random_state=42, stratify=y_def
)

# Train Random Forest (multi-class)
clf_stage2 = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
clf_stage2.fit(X_train, y_train)

# Evaluate
y_pred = clf_stage2.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Stage-2 Accuracy:", acc)

print("\nStage-2 Classification Report:")
print(classification_report(y_test, y_pred))

print("\nStage-2 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Deforested feature matrix shape: (1764, 1280)
Subtype labels shape: (1764,)
Unique subtypes: [0 2 3 4 5 6 7 8 9]
Stage-2 Accuracy: 0.8328611898016998

Stage-2 Classification Report:
              precision    recall  f1-score   support

           0       0.76      1.00      0.86        48
           2       0.81      0.80      0.80        44
           3       0.80      0.57      0.67        35
           4       0.87      0.87      0.87        38
           5       0.95      0.70      0.81        30
           6       0.81      0.81      0.81        31
           7       0.89      0.98      0.93        43
           8       0.68      0.74      0.71        38
           9       0.98      0.91      0.94        46

    accuracy                           0.83       353
   macro avg       0.84      0.82      0.82       353
weighted avg       0.84      0.83      0.83       353


Stage-2 Confusion Matrix:
[[48  0  0  0  0  0  0  0  0]
 [ 0 35  0  1  0  5  0  2  1]
 [ 2  3 20  1  0  0  1  8 

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# Reuse your pretrained feature extractor
def extract_features(img_path, base_model):
    img = image.load_img(img_path, target_size=(128,128))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x, verbose=0)
    return features.flatten().reshape(1, -1)

# Pipeline function
def predict_pipeline(img_path, base_model, clf_stage1, clf_stage2, subtype_mapping):
    # Extract features
    feat = extract_features(img_path, base_model)

    # Stage 1: Forest vs. Deforested
    stage1_pred = clf_stage1.predict(feat)[0]
    if stage1_pred == 0:
        return "Forest"
    else:
        # Stage 2: Which type of deforested
        subtype_pred = clf_stage2.predict(feat)[0]
        subtype_name = subtype_mapping.get(subtype_pred, f"Subtype-{subtype_pred}")
        return f"Deforested → {subtype_name}"


In [ ]:
# Testing

test_img1= "/content/drive/MyDrive/Colab Notebooks/EuroSAT/Deforested/Residential/Residential_2.jpg"
test_img2= "/content/drive/MyDrive/Colab Notebooks/EuroSAT/Deforested/Industrial/Industrial_9.jpg"

result = predict_pipeline(test_img2, base_model, clf_stage1, clf_stage2, subtype_mapping)
print("Prediction:", result)


Prediction: Deforested → Industrial


In [ ]:
import joblib

joblib.dump(clf_stage1, "/content/drive/MyDrive/Colab Notebooks/clf_stage1.pkl")
joblib.dump(clf_stage2, "/content/drive/MyDrive/Colab Notebooks/clf_stage2.pkl")

print("✅ Models saved as clf_stage1.pkl and clf_stage2.pkl")


✅ Models saved as clf_stage1.pkl and clf_stage2.pkl


In [ ]:
import os

# Check if file exists
print(os.path.exists("/content/drive/MyDrive/Colab Notebooks/clf_stage1.pkl"))
print(os.path.exists("/content/drive/MyDrive/Colab Notebooks/clf_stage2.pkl"))

# List contents of folder
!ls -lh "/content/drive/MyDrive/Colab Notebooks"

True
True
total 15M
-rw------- 1 root root 1.6M Aug 30 13:33  clf_stage1.pkl
-rw------- 1 root root  13M Aug 30 13:33  clf_stage2.pkl
-rw------- 1 root root  14K Aug 30 13:40  deforesration_Detect.py
-rw------- 1 root root  16K Aug 30 13:40 'Deforestion Random forest.ipynb'
drwx------ 4 root root 4.0K Aug 30 05:37  EuroSAT
-rw------- 1 root root 1.2K Aug 30 13:21  Untitled0.ipynb


In [ ]:
!pip install streamlit
import streamlit as st
import numpy as np
import os
from PIL import Image
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import joblib   # if you want to save/load your models

# -------------------------
# Load pretrained CNN
# -------------------------
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

# Load trained classifiers (after you save them with joblib.dump)
# clf_stage1 = joblib.load("clf_stage1.pkl")
# clf_stage2 = joblib.load("clf_stage2.pkl")

# For demo – assuming you already have them in memory
from sklearn.ensemble import RandomForestClassifier
clf_stage1 = joblib.load("/content/drive/MyDrive/Colab Notebooks/clf_stage1.pkl")
clf_stage2 = joblib.load("/content/drive/MyDrive/Colab Notebooks/clf_stage2.pkl")



# Mapping (make sure to use your actual mapping dictionary)
subtype_mapping = {
    0: "AnnualCrop",
    2: "HerbaceousVegetation",
    3: "Industrial",
    4: "Pasture",
    5: "PermanentCrop",
    6: "Residential",
    7: "River",
    8: "SeaLake",
    9: "Highway"
}

# -------------------------
# Feature extraction function
# -------------------------
def extract_features(img):
    img = img.resize((128, 128))   # same as training
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = base_model.predict(x, verbose=0)
    return features.flatten()

# -------------------------
# Prediction pipeline
# -------------------------
def predict_pipeline(uploaded_img):
    # Extract features
    features = extract_features(uploaded_img).reshape(1, -1)

    # Stage-1: Forest vs Deforested
    stage1_pred = clf_stage1.predict(features)[0]

    if stage1_pred == 0:
        return "🌲 Forest"
    else:
        # Stage-2: Subtype classification
        subtype_pred = clf_stage2.predict(features)[0]
        subtype_name = subtype_mapping.get(subtype_pred, "Unknown")
        return f"🌾 Deforested → {subtype_name}"

# -------------------------
# Streamlit UI
# -------------------------
st.set_page_config(page_title="Deforestation Classifier", page_icon="🌍")
st.title("🛰️ Deforestation Detection")
st.write("Upload a satellite image to classify whether it's **Forest** or **Deforested**, and if deforested → subtype.")

uploaded_file = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = Image.open(uploaded_file).convert("RGB")
    st.image(img, caption="Uploaded Image", use_column_width=True)

    if st.button("🔍 Predict"):
        result = predict_pipeline(img)
        st.success(f"Prediction: {result}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.4 MB/s eta 0:00:00


/tmp/ipython-input-4091904811.py:13: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
2025-08-30 13:42:03.855 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-30 13:42:03.856 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-30 13:42:04.039 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-30 13:42:04.040 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-30 13:42:04.041 Thread 'MainThread': missing ScriptRunContext! This

SyntaxError: invalid syntax (ipython-input-1609277159.py, line 1)